In [3]:
import torch
import os
from torch import nn
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm
import numpy as np




class Tetris_shapes(nn.Module):
    def __init__(self,input_features, output_features, hidden_units):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(nn.Linear(in_features=input_features, out_features=hidden_units),
                                                nn.ReLU(),
                                                nn.Linear(in_features=hidden_units, out_features=hidden_units),
                                                nn.ReLU(),
                                                nn.Linear(in_features=hidden_units, out_features=output_features))

        for layer in self.linear_layer_stack:
          if isinstance(layer, nn.Linear):
            nn.init.kaiming_uniform_(layer.weight)



    def forward(self, x):
        return self.linear_layer_stack(x)


model_linear_correlated_data = Tetris_shapes(input_features=64, output_features=2, hidden_units=10)

model_linear_correlated_data_dict = torch.load("/content/linear_1d1p_0.0125_correlated.pt")


def collect_activation(model,x):
  x= torch.tensor(np.array(x, dtype=np.float32))
  with torch.no_grad():
    layers = [model.linear_layer_stack[i] for i in range(len(model.linear_layer_stack))]
    activation = []
    activation.append(x)
    for layer in layers:
      x = layer(x)
      activation.append(x)
  return activation

#This function trains a given model in batches and print out the results at every 10th epoch
def training_model_dicts(model: torch.nn.Module, model_dict: dict, epochs : int = 10):

    x_train = torch.tensor(np.array(model_dict["x_train"], dtype=np.float32))
    y_train = torch.tensor(np.array(model_dict["y_train"], dtype=np.int64))
    x_test = torch.tensor(np.array(model_dict["x_val"], dtype=np.float32))
    y_test = torch.tensor(np.array(model_dict["y_val"], dtype=np.int64))


    train_dataset = TensorDataset(x_train, y_train)
    test_dataset = TensorDataset(x_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []
             }


    for epochs in tqdm(range(epochs)):
        model.train()
        train_loss = 0
        train_acc = 0
        for batch ,(x_batch, y_batch) in enumerate(train_loader):
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            train_acc += (y_pred_class == y_batch).sum().item() / len(y_pred)

        train_loss /= len(train_loader)
        train_acc /= len(train_loader)

        model.eval()
        test_loss = 0
        test_acc = 0
        with torch.inference_mode():
            total_loss = 0
            for batch ,(x_batch, y_batch) in enumerate(test_loader):
                test_y_pred = model(x_batch)
                loss = loss_fn(test_y_pred, y_batch)
                test_loss += loss.item()
                y_pred_class = torch.argmax(torch.softmax(test_y_pred, dim=1), dim=1)
                test_acc += (y_pred_class == y_batch).sum().item() / len(test_y_pred)

            test_loss /= len(test_loader)
            test_acc /= len(test_loader)

        if(epochs % 10 == 0):
          print(
          f"Epoch: {epochs} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
          )


        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results



<ipython-input-3-41e095c7845a>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_linear_correlated_data_dict = torch.load("/content/linear_1d1p_0.0125_correlated.pt")


In [4]:
training_model_dicts(model=model_linear_correlated_data, model_dict=model_linear_correlated_data_dict, epochs=100)
acts = collect_activation(model=model_linear_correlated_data,x=model_linear_correlated_data_dict["x_train"])

acts_dict = {"activations": acts}
acts_dict['layers'] = [['input'], ['linear', 'relu'], ['linear', 'relu'], ['linear']]
torch.save(acts_dict,"acts.pt")
torch.save(model_linear_correlated_data.state_dict(),"linear_correlated_model.pt")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0 | train_loss: 0.6965 | train_acc: 0.5034 | test_loss: 0.6907 | test_acc: 0.5195
Epoch: 10 | train_loss: 0.3629 | train_acc: 0.9795 | test_loss: 0.3081 | test_acc: 1.0000
Epoch: 20 | train_loss: 0.0120 | train_acc: 1.0000 | test_loss: 0.0100 | test_acc: 1.0000
Epoch: 30 | train_loss: 0.0015 | train_acc: 1.0000 | test_loss: 0.0013 | test_acc: 1.0000
Epoch: 40 | train_loss: 0.0006 | train_acc: 1.0000 | test_loss: 0.0005 | test_acc: 1.0000
Epoch: 50 | train_loss: 0.0003 | train_acc: 1.0000 | test_loss: 0.0003 | test_acc: 1.0000
Epoch: 60 | train_loss: 0.0001 | train_acc: 1.0000 | test_loss: 0.0001 | test_acc: 1.0000
Epoch: 70 | train_loss: 0.0000 | train_acc: 1.0000 | test_loss: 0.0000 | test_acc: 1.0000
Epoch: 80 | train_loss: 0.0000 | train_acc: 1.0000 | test_loss: 0.0000 | test_acc: 1.0000
Epoch: 90 | train_loss: 0.0000 | train_acc: 1.0000 | test_loss: 0.0000 | test_acc: 1.0000


In [5]:
def add_targets(activation_dict,model_data_dict):
  activation_dict["targets"] = model_data_dict["y_train"]
  return activation_dict

acts_file = torch.load("acts.pt")
act_dict = add_targets(acts_file,model_linear_correlated_data_dict)

torch.save(act_dict, "acts_targets.pt")

<ipython-input-5-ec9b47e664aa>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  acts_file = torch.load("acts.pt")


In [10]:
import torch

def add_alt_node_node_key(random_acts_dict, model):
  """Adds a 'node_node' key to the dictionary with alternating 1 and -1 values structured like the model's weights."""

  node_node_list = []
  current_val = 1  # Start with 1

  # Iterate through model layers to get weight shapes
  for name, param in model.named_parameters():
    if 'weight' in name:
      weight_shape = param.shape  # Get the shape of the weight
      node_node_list.append(torch.ones(*weight_shape) * current_val)  # Fill with current_val

      current_val *= -1  # Alternate between 1 and -1

  random_acts_dict["node_node"] = node_node_list  # Add the new key
  return random_acts_dict


# Load the dictionary (if not already loaded)
acts_targets_file = torch.load("acts_targets.pt")

# Add the "node_node" key
alt_node_node_act =add_alt_node_node_key(acts_targets_file,model_linear_correlated_data)

torch.save(alt_node_node_act, "acts_targets_node_node.pt")

<ipython-input-10-4dbc97783080>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  acts_targets_file = torch.load("acts_targets.pt")
